In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Session-based Recommendation with XLNET

In this notebook, we build a session-based recommendation model with XLNET, and train and evaluate it with NVTabular Pytorch Dataloader. 

In [2]:
import os
import glob
import torch 

## Build a DL model with Transformers4Rec library  

- import required libraries

In [3]:
import transformers4rec.torch as torch4rec
from transformers4rec.torch import TabularSequenceFeatures, MLPBlock, SequentialBlock, Head, TransformerBlock

from transformers4rec.utils.schema import DatasetSchema
from transformers4rec.torch.model.head import NextItemPredictionTask
from transformers4rec.config import transformer
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt

- Manually set the schema 

In [4]:
# Define schema object to pass it to the SequentialTabularFeatures
schema = DatasetSchema.from_proto("schema.pb")

# select the features to be fed to model to train
schema = schema.select_by_name(['item_id-list_trim', 'category-list_trim', 'timestamp/weekday/sin-list_trim','timestamp/age_days-list_trim'])

### Define the sequential input module

Below we define our `input` bloc using [`SequentialTabularFeatures` class](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/features/sequential.py). The `from_schema` module directly parse schema and accepts categorical and continuous sequential inputs and supports data augmentation, data aggregation, `sequential-concat` and `elementwise-sum` aggregations, the projection of the interaction embeddings and the masking tasks.

`max_sequence_length` defines the maximum sequence length of our sequential input, and if `continuous_projection` argument is set,  all numerical features are concatenated and projected by a number of MLP layers.

In [5]:
inputs = TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=20,
        continuous_projection=64,
        d_output=100,
        masking="causal",
    )
inputs.masking.device = 'cuda'

- LM task + HF Transformer architecture + Next item-prediction task. 
    - We build a [T4RecConfig](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/config/transformer.py#L8) class to update the config class of the transformer architecture with the specified arguments, then load the related model. Here we use it to instantiate an XLNET model according to the  arguments (d_model, n_head, etc.), defining the model architecture.
    - [TransformerBlock](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/block/transformer.py#L37) class is created to support HF Transformers for session-based and sequential-based recommendation models.
    - [NextItemPredictionTask](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/transformers4rec/torch/model/head.py#L238) is the class to support next item prediction task.

In [6]:
# Define XLNetConfig class and set default parameters 

# Set HF config of XLNet 
transformer_config = transformer.XLNetConfig.build(
    d_model=64, n_head=4, n_layer=2, total_seq_length=20
)
# Define the model block including: inputs, masking, projection and transformer block.
body = torch4rec.SequentialBlock(
    inputs, torch4rec.MLPBlock([64]), torch4rec.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the head related to next item prediction task 
head = torch4rec.Head(
    body,
    torch4rec.NextItemPredictionTask(weight_tying=True, hf_format=True),
    inputs=inputs,
)

# Get the end-to-end Model class 
model = torch4rec.Model(head)

### Train the model 

As data loader we use optimized NVTabular PyTorch Dataloader. You can learn more about it [here](https://nvidia.github.io/NVTabular/main/training/pytorch.html).

- **Set Training arguments**

In [7]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
#Set argumentd for training 
train_args = T4RecTrainingArguments(local_rank = -1, dataloader_drop_last = True, data_loader_engine='nvtabular',
                                  report_to = [], debug = ["r"], gradient_accumulation_steps = 32,
                                  per_device_train_batch_size = 256, per_device_eval_batch_size = 32,
                                  output_dir = ".", lr_scheduler_type='cosine', 
                                  learning_rate_num_cosine_cycles_by_epoch=1.5,
                                  max_sequence_length=20, fp16=False, no_cuda=False)

# Daily Fine-Tuning: Training over a time window

Here we do daily fine-tuning meaning that we use the first day to train and second day to evaluate, then we use the second day data to train the model by resuming from the first step, and evaluate on the third day, so on so forth.

In [8]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

In [9]:
start_time_window_index = 1
final_time_window_index = 7
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(f"/workspace/data/sessions_by_day/{time_index_train}/train.parquet")
    eval_paths = glob.glob(f"/workspace/data/sessions_by_day/{time_index_eval}/valid.parquet")  
    
    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.train_dataset = train_paths
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    # Evaluate on the following day
    trainer.eval_dataset = eval_paths
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 
    trainer.wipe_memory()

********************
Launch training for day 1 are:
********************



***** Running training *****
  Num examples = 768
  Num Epochs = 3
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 32768
  Gradient Accumulation steps = 32
  Total optimization steps = 3


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




***** Running training *****
  Num examples = 768
  Num Epochs = 3
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 32768
  Gradient Accumulation steps = 32
  Total optimization steps = 3


...



Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running training *****
  Num examples = 768
  Num Epochs = 3
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 32768
  Gradient Accumulation steps = 32
  Total optimization steps = 3


********************
Eval results for day 6 are:	

********************

 epoch = 3.0
 eval_avgprecisionat_10 = 0.0
 eval_avgprecisionat_20 = 5.771006544819102e-05
 eval_loss = 10.945877075195312
 eval_ndcgat_10 = 0.0
 eval_ndcgat_20 = 0.00025370417279191315
 eval_recallat_10 = 0.0
 eval_recallat_20 = 0.0010964912362396717
 eval_runtime = 0.1803
 eval_samples_per_second = 532.311
 eval_steps_per_second = 5.545
********************
Launch training for day 6 are:
********************



Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




********************
Eval results for day 7 are:	

********************

 epoch = 3.0
 eval_avgprecisionat_10 = 0.0
 eval_avgprecisionat_20 = 0.0
 eval_loss = 10.941803932189941
 eval_ndcgat_10 = 0.0
 eval_ndcgat_20 = 0.0
 eval_recallat_10 = 0.0
 eval_recallat_20 = 0.0
 eval_runtime = 0.1791
 eval_samples_per_second = 535.94
 eval_steps_per_second = 5.583


* **Save the model**

In [10]:
trainer._save_model_and_checkpoint(save_model_class=True)

Saving model checkpoint to ./checkpoint-4
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


* **Reload the model**

In [11]:
trainer.load_model_trainer_states_from_checkpoint('./checkpoint-%s'%trainer.state.global_step)

- **Re-compute eval metrics of validation data**

In [16]:
eval_data_paths = glob.glob(f"/workspace/data/sessions_by_day/7/valid.parquet")  

In [18]:
# set new data from last day
eval_metrics = trainer.evaluate(eval_dataset=eval_data_paths, metric_key_prefix='eval')
for key in sorted(eval_metrics.keys()):
    print("  %s = %s" % (key, str(eval_metrics[key])))

  epoch = 3.0
  eval_avgprecisionat_10 = 0.0
  eval_avgprecisionat_20 = 0.0
  eval_loss = 10.941803932189941
  eval_ndcgat_10 = 0.0
  eval_ndcgat_20 = 0.0
  eval_recallat_10 = 0.0
  eval_recallat_20 = 0.0
  eval_runtime = 0.1865
  eval_samples_per_second = 514.87
  eval_steps_per_second = 5.363
